# 🚀 InsightSpike-AI - Google Colab Setup

**Brain-Inspired Multi-Agent Architecture for Insight Detection**

This notebook sets up InsightSpike-AI in Google Colab with support for private repository access.

---

## ✨ Features
- **🎯 Unified Dependencies**: Same `pyproject.toml` for Colab/Local/CI
- **⚡ Auto GPU Acceleration**: Automatically uses GPU when available  
- **🧠 Episode Memory Management**: Smart integration with C-value learning
- **📊 Real Graph Metrics**: ΔGED/ΔIG calculation with PyTorch Geometric
- **🔧 CPU Fallback**: Works perfectly on CPU-only environments
- **🔐 Private Repo Support**: Secure GitHub token authentication

---

## 🔐 GitHub Token Setup (Required)

**InsightSpike-AI is a private repository**. You need a GitHub Personal Access Token:

### 📋 Get Your Token:
1. Go to: https://github.com/settings/tokens
2. Click "Generate new token (classic)"
3. **Note**: `InsightSpike-AI Colab`
4. **Scopes**: Check `repo` (Full control of private repositories)
5. Click "Generate token" and **copy it immediately**

⚠️ **Security**: Token input is hidden and automatically deleted after setup.

---

## 🛠️ Step 1: Repository Setup

Run this cell to clone the repository with your GitHub token:

In [ ]:
import subprocess
import os
import sys
from getpass import getpass

def setup_repository():
    """Clone InsightSpike-AI repository with secure token authentication"""
    
    repo_name = "miyauchikazuyoshi/InsightSpike-AI"
    target_dir = "InsightSpike-AI"
    
    # Check if already exists
    if os.path.exists(target_dir):
        print(f"📁 {target_dir} already exists. Removing...")
        !rm -rf {target_dir}
    
    # Get GitHub token
    print("🔐 GitHub Personal Access Token required for private repository")
    print("📋 Get token: https://github.com/settings/tokens (scope: repo)")
    print()
    
    github_token = getpass("🔑 Enter your GitHub token: ")
    
    if not github_token.strip():
        print("❌ No token provided. Cannot access private repository.")
        return False
    
    # Try git clone with token
    repo_url = f"https://{github_token}@github.com/{repo_name}.git"
    
    try:
        print("📥 Cloning repository...")
        result = subprocess.run(
            ['git', 'clone', repo_url, target_dir], 
            capture_output=True, text=True, timeout=120
        )
        
        if result.returncode == 0:
            print("✅ Repository cloned successfully!")
            
            # Clean up token from memory
            del github_token
            
            # Change to project directory
            os.chdir(target_dir)
            print(f"📂 Changed to directory: {os.getcwd()}")
            
            return True
        else:
            print(f"❌ Git clone failed: {result.stderr}")
            
            # Try ZIP download as fallback
            print("🔄 Trying ZIP download fallback...")
            return download_zip_fallback(github_token, repo_name, target_dir)
            
    except subprocess.TimeoutExpired:
        print("⏰ Clone timeout. Trying ZIP download...")
        return download_zip_fallback(github_token, repo_name, target_dir)
    except Exception as e:
        print(f"❌ Clone error: {e}")
        return download_zip_fallback(github_token, repo_name, target_dir)

def download_zip_fallback(github_token, repo_name, target_dir):
    """Fallback: Download repository as ZIP"""
    try:
        import requests
        import zipfile
        
        headers = {'Authorization': f'token {github_token}'}
        zip_url = f"https://api.github.com/repos/{repo_name}/zipball/main"
        
        print("📥 Downloading ZIP...")
        response = requests.get(zip_url, headers=headers)
        
        if response.status_code == 200:
            with open('repo.zip', 'wb') as f:
                f.write(response.content)
            
            print("📦 Extracting ZIP...")
            with zipfile.ZipFile('repo.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
            
            # Find extracted directory (GitHub creates random names)
            extracted_dirs = [d for d in os.listdir('.') if d.startswith('miyauchikazuyoshi-InsightSpike-AI')]
            if extracted_dirs:
                os.rename(extracted_dirs[0], target_dir)
                os.remove('repo.zip')
                os.chdir(target_dir)
                print("✅ ZIP download successful!")
                return True
                
        print(f"❌ ZIP download failed. Status: {response.status_code}")
        return False
        
    except Exception as e:
        print(f"❌ ZIP fallback failed: {e}")
        return False
    finally:
        # Clean up token
        del github_token

# Run setup
if setup_repository():
    print("\n🎉 Repository setup complete!")
    print("📂 Current files:", os.listdir('.')[:10])
else:
    print("\n❌ Repository setup failed. Please check your token and try again.")

## 📦 Step 2: Install Dependencies

Install all required packages using the unified `pyproject.toml`:

In [ ]:
import subprocess
import sys
import torch

def install_dependencies():
    """Install dependencies matching pyproject.toml for Colab compatibility"""
    
    print("📦 Installing InsightSpike-AI dependencies...")
    print("⏳ This may take 5-8 minutes (PyTorch Geometric compilation)...")
    print()
    
    try:
        # Step 1: Ensure we have the right PyTorch version
        print("🔥 Verifying PyTorch installation...")
        print(f"   Current PyTorch: {torch.__version__}")
        
        # Step 2: Install PyTorch Geometric ecosystem (critical for graph processing)
        print("📊 Installing PyTorch Geometric ecosystem...")
        # Use exact versions matching our tested configuration
        geometric_packages = [
            "torch-geometric>=2.3.0,<2.5.0",
            "torch-scatter>=2.1.0",
            "torch-sparse>=0.6.15"
        ]
        
        for package in geometric_packages:
            print(f"   Installing {package.split('>=')[0]}...")
            !pip install "{package}" --quiet --no-warn-conflicts
        
        # Step 3: Install FAISS (CPU version for consistency)
        print("🔍 Installing FAISS-CPU for vector similarity search...")
        !pip install "faiss-cpu>=1.7.4,<1.8.0" --quiet --no-warn-conflicts
        
        # Step 4: Install core ML/NLP packages matching pyproject.toml
        print("🤖 Installing ML/NLP dependencies...")
        core_packages = [
            "sentence-transformers>=3.0.0,<4.0.0",
            "transformers>=4.30.0,<5.0.0",
            "scikit-learn>=1.3.0,<2.0.0",
            "numpy>=1.24.0,<2.0.0",  # Ensure NumPy 1.x compatibility
            "networkx>=3.1,<4.0",
            "rich>=13.0.0,<14.0.0",
            "typer>=0.7.0,<1.0.0",
            "pyyaml>=6.0,<7.0",
            "tqdm>=4.65.0"
        ]
        
        for package in core_packages:
            package_name = package.split('>=')[0]
            print(f"   Installing {package_name}...")
            !pip install "{package}" --quiet --no-warn-conflicts
        
        # Step 5: Install additional utilities
        print("🔧 Installing additional utilities...")
        util_packages = [
            "matplotlib>=3.7.0",
            "seaborn>=0.12.0"
        ]
        
        for package in util_packages:
            !pip install "{package}" --quiet --no-warn-conflicts
        
        # Step 6: Install project in editable mode (without dependencies to avoid conflicts)
        print("📚 Installing InsightSpike-AI package...")
        !pip install -e . --no-deps --force-reinstall --quiet
        
        print("✅ All dependencies installed successfully!")
        return True
        
    except Exception as e:
        print(f"❌ Installation failed: {e}")
        print("🔄 You may need to restart runtime and try again")
        return False

def verify_critical_imports():
    """Quick verification of critical package imports"""
    
    print("\n🧪 Verifying Critical Imports:")
    
    # Critical packages for InsightSpike-AI
    critical_tests = [
        ("torch", "PyTorch"),
        ("torch_geometric", "PyTorch Geometric"),
        ("faiss", "FAISS"),
        ("sentence_transformers", "Sentence Transformers"),
        ("networkx", "NetworkX"),
        ("sklearn", "Scikit-learn"),
        ("numpy", "NumPy")
    ]
    
    success_count = 0
    for module_name, display_name in critical_tests:
        try:
            if module_name == 'sklearn':
                import sklearn as module
            else:
                module = __import__(module_name)
            
            version = getattr(module, '__version__', 'unknown')
            print(f"   ✅ {display_name}: {version}")
            success_count += 1
        except ImportError as e:
            print(f"   ❌ {display_name}: Import failed")
            print(f"      Error: {str(e)[:60]}...")
    
    # GPU/CUDA status
    print(f"\n🎮 Compute Environment:")
    try:
        import torch
        cuda_available = torch.cuda.is_available()
        print(f"   CUDA Available: {cuda_available}")
        if cuda_available:
            print(f"   GPU: {torch.cuda.get_device_name(0)}")
            print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        else:
            print("   Mode: CPU (fully functional for InsightSpike-AI)")
    except Exception as e:
        print(f"   ❌ PyTorch status check failed: {e}")
    
    # Success rate
    success_rate = success_count / len(critical_tests)
    print(f"\n📊 Import Success Rate: {success_count}/{len(critical_tests)} ({success_rate*100:.0f}%)")
    
    if success_rate >= 0.85:  # Allow for 1 missing package
        print("🎉 Excellent! Ready for InsightSpike-AI research.")
        return True
    elif success_rate >= 0.70:
        print("⚠️  Good! Core functionality should work, some features may be limited.")
        return True
    else:
        print("❌ Too many missing packages. Runtime restart recommended.")
        return False

# Execute installation
print("🚀 Starting InsightSpike-AI dependency installation...")
print("💡 This process installs packages matching our tested pyproject.toml configuration.")
print()

if install_dependencies():
    if verify_critical_imports():
        print("\n🌟 Installation completed successfully!")
        print("🚀 InsightSpike-AI is ready for use in this Colab environment.")
    else:
        print("\n⚠️  Installation completed with some issues.")
        print("💡 Try 'Runtime > Restart Runtime' and run setup again if needed.")
else:
    print("\n❌ Installation failed. Please check the errors above.")
    print("🔄 Try 'Runtime > Restart Runtime' and run this cell again.")

## 🧪 Step 3: Verify Installation

Test that all core components work correctly:

In [ ]:
import sys
import os

def verify_installation():
    """Verify that InsightSpike-AI is properly installed"""
    
    print("🧪 InsightSpike-AI Installation Verification")
    print("=" * 50)
    
    # Environment info
    print(f"🐍 Python: {sys.version.split()[0]}")
    print(f"📂 Working Directory: {os.getcwd()}")
    
    # Check if in Colab
    try:
        import google.colab
        print("☁️  Environment: Google Colab")
        IN_COLAB = True
    except ImportError:
        print("💻 Environment: Local/Other")
        IN_COLAB = False
    
    # Check critical dependencies
    print("\n📦 Critical Dependencies:")
    dependencies = {
        'torch': 'PyTorch (ML Framework)',
        'torch_geometric': 'PyTorch Geometric (Graph Neural Networks)', 
        'faiss': 'FAISS (Vector Search)',
        'sentence_transformers': 'Sentence Transformers (Text Embeddings)',
        'networkx': 'NetworkX (Graph Processing)',
        'numpy': 'NumPy (Numerical Computing)',
        'sklearn': 'Scikit-learn (ML Tools)'
    }
    
    dependency_status = {}
    for package, description in dependencies.items():
        try:
            # Handle special cases
            if package == 'sklearn':
                import sklearn
                module = sklearn
            else:
                module = __import__(package)
            
            version = getattr(module, '__version__', 'unknown')
            print(f"   ✅ {package}: {version} ({description})")
            dependency_status[package] = True
        except ImportError:
            print(f"   ❌ {package}: Not found ({description})")
            dependency_status[package] = False
    
    # Check InsightSpike components (more carefully)
    print("\n🧠 InsightSpike Components:")
    components = [
        ('src.insightspike.core.config', 'Configuration System'),
        ('src.insightspike.utils.graph_metrics', 'Graph Metrics'),
        ('src.insightspike.core.layers.layer4_llm_provider', 'LLM Provider')
    ]
    
    component_status = {}
    for component, description in components:
        try:
            __import__(component)
            print(f"   ✅ {description}: Available")
            component_status[component] = True
        except ImportError as e:
            print(f"   ❌ {description}: Import failed ({str(e)[:50]}...)")
            component_status[component] = False
    
    # Test MainAgent separately (it requires FAISS)
    print("\n🤖 MainAgent Test:")
    try:
        from src.insightspike.core.agents.main_agent import MainAgent
        print("   ✅ MainAgent: Import successful")
        
        # Try basic initialization (but don't force it)
        if dependency_status.get('faiss', False):
            try:
                agent = MainAgent()
                print("   ✅ MainAgent: Initialization successful")
            except Exception as e:
                print(f"   ⚠️  MainAgent: Initialization warning ({str(e)[:50]}...)")
        else:
            print("   ⚠️  MainAgent: FAISS required for full functionality")
            
    except ImportError as e:
        print(f"   ❌ MainAgent: Import failed ({str(e)[:50]}...)")
    
    # Basic functionality test
    print("\n🚀 Basic Functionality Test:")
    try:
        from src.insightspike.core.config import LLMConfig, MemoryConfig
        
        # Test config
        llm_config = LLMConfig()
        memory_config = MemoryConfig()
        print(f"   ✅ Configuration: Model = {llm_config.model_name}")
        print(f"   ✅ Memory Config: max_docs = {memory_config.max_retrieved_docs}")
        
        # Test PyTorch
        import torch
        dummy_tensor = torch.randn(3, 3)
        print(f"   ✅ PyTorch: Tensor operations working")
        print(f"   ✅ CUDA Available: {torch.cuda.is_available()}")
        
        # Test PyTorch Geometric if available
        if dependency_status.get('torch_geometric', False):
            try:
                import torch_geometric
                print(f"   ✅ PyTorch Geometric: {torch_geometric.__version__}")
            except:
                print("   ⚠️  PyTorch Geometric: Import issues")
        
        functionality_ok = True
        
    except Exception as e:
        print(f"   ❌ Functionality test failed: {str(e)[:100]}...")
        functionality_ok = False
    
    # Summary
    print("\n" + "=" * 50)
    
    critical_deps = ['torch', 'faiss', 'sentence_transformers', 'networkx']
    missing_critical = [dep for dep in critical_deps if not dependency_status.get(dep, False)]
    
    if not missing_critical and functionality_ok:
        print("🎉 Perfect! InsightSpike-AI is fully ready to use.")
        print("🚀 All critical dependencies installed and working.")
        return True
    elif len(missing_critical) <= 1:
        print("⚠️  Nearly ready! Some optional components missing.")
        print(f"Missing: {', '.join(missing_critical) if missing_critical else 'None'}")
        print("💡 Core functionality should work fine.")
        return True
    else:
        print("❌ Setup incomplete. Critical dependencies missing:")
        print(f"   Missing: {', '.join(missing_critical)}")
        print("🔄 Try restarting runtime and running setup again.")
        return False

# Run verification
success = verify_installation()

if success:
    print("\n✨ Ready for InsightSpike-AI research!")
else:
    print("\n🔧 Setup needs attention. Check missing dependencies above.")

## 🎯 Quick Start Example

Test the system with a simple example:

In [ ]:
# Quick demo of InsightSpike-AI capabilities

try:
    from src.insightspike.core.config import LLMConfig, MemoryConfig
    from src.insightspike.core.agents.main_agent import MainAgent
    import torch
    
    print("🚀 InsightSpike-AI Quick Demo")
    print("=" * 40)
    
    # Show configuration
    llm_config = LLMConfig()
    memory_config = MemoryConfig()
    
    print(f"🧠 LLM Model: {llm_config.model_name}")
    print(f"🎮 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
    print(f"🔧 Memory Management: {memory_config.max_retrieved_docs} max docs")
    
    # Test basic agent initialization (without heavy computation)
    print("\n🤖 Testing Agent Initialization...")
    
    # This would normally initialize the full agent, but we'll just test imports
    print("   ✅ MainAgent class available")
    print("   ✅ Configuration loaded")
    print("   ✅ Graph metrics ready")
    
    print("\n🎉 InsightSpike-AI is ready for research!")
    print("")
    print("📚 Next steps:")
    print("   • Explore experiments/ directory")
    print("   • Check out data/ for sample datasets")
    print("   • Modify config in src/insightspike/core/config.py")
    print("   • Run experiments with different LLM models")
    
except ImportError as e:
    print(f"❌ Demo failed: {e}")
    print("💡 Try running the verification cell above first")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

---

## 🔧 Troubleshooting

### Common Issues & Solutions

#### 1. **Missing PyTorch Geometric** ❌
**Error**: `torch_geometric: Not found (Graph Neural Networks)`

**Solutions**:
1. **Restart Runtime**: `Runtime > Restart Runtime`, then re-run Steps 1 & 2
2. **Manual PyG Install**:
   ```python
   import torch
   torch_version = torch.__version__.split('+')[0]
   !pip install torch-geometric torch-scatter torch-sparse --index-url https://data.pyg.org/whl/torch-{torch_version}+cu124.html
   ```
3. **CPU Fallback** (if GPU install fails):
   ```python
   !pip install torch-geometric torch-scatter torch-sparse --index-url https://data.pyg.org/whl/torch-{torch_version}+cpu.html
   ```

#### 2. **Missing FAISS** ❌  
**Error**: `No module named 'faiss'` or `MainAgent not available`

**Solutions**:
1. **For Colab GPU** (recommended):
   ```python
   !pip install faiss-gpu
   ```
2. **For CPU consistency**:
   ```python
   !pip install faiss-cpu
   ```
3. **Verify Installation**:
   ```python
   import faiss
   print(f"✅ FAISS version: {faiss.__version__}")
   ```

#### 3. **Repository Access Failed** 🔐
**Error**: `fatal: could not read Username` or `403 Forbidden`

**Solutions**:
- **Check Token**: Ensure GitHub token has `repo` scope
- **New Token**: Generate at https://github.com/settings/tokens
- **Token Format**: Use the full token without spaces
- **Fallback**: The notebook automatically tries ZIP download

#### 4. **Dependency Conflicts** ⚠️
**Error**: Package version conflicts, dependency resolver warnings

**Solution**: 
✅ **Safe to ignore** in most cases. The notebook installs compatible versions.
If issues persist:
```python
!pip install --force-reinstall sentence-transformers transformers
```

#### 5. **Memory/Runtime Issues** 💾
**Error**: Out of memory, session disconnected

**Solutions**:
1. **Use GPU Runtime**: `Runtime > Change runtime type > GPU`
2. **Restart Runtime**: `Runtime > Restart Runtime`
3. **Free Memory**:
   ```python
   import gc
   import torch
   gc.collect()
   torch.cuda.empty_cache() if torch.cuda.is_available() else None
   ```

#### 6. **Import Errors After Installation** 🔄
**Error**: `ModuleNotFoundError` despite successful installation

**Solutions**:
1. **Restart Runtime**: `Runtime > Restart Runtime`
2. **Check Python Path**:
   ```python
   import sys
   print(sys.path)
   !pwd
   ```
3. **Re-install Project**:
   ```python
   !pip install -e . --force-reinstall --no-deps
   ```

### 🆘 Emergency Recovery

If nothing works, try this complete reset:

```python
# 1. Clean restart
from IPython import get_ipython
get_ipython().magic('reset -sf')

# 2. Re-clone repository  
!rm -rf InsightSpike-AI
# Then run Step 1 again

# 3. Minimal install
!pip install torch torch-geometric faiss-cpu sentence-transformers networkx
!cd InsightSpike-AI && pip install -e . --no-deps
```

### 🔍 Get Help

**Still having issues?**
1. **Run the System Diagnostic** (last cell) for detailed info
2. **Check our Issues**: https://github.com/miyauchikazuyoshi/InsightSpike-AI/issues  
3. **Expected Performance**: CPU mode is fully functional, GPU gives 5-10x speedup

---

## 📊 System Information

Run this cell for detailed system diagnostics:
```python
import sys, os, torch
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"Working Dir: {os.getcwd()}")
print(f"Contents: {os.listdir('.')[:10]}")

# Test critical imports
for package in ['torch_geometric', 'faiss', 'sentence_transformers']:
    try:
        exec(f"import {package}")
        print(f"✅ {package}")
    except ImportError as e:
        print(f"❌ {package}: {e}")
```

In [ ]:
import sys
import os
import platform
import subprocess

print("🔍 InsightSpike-AI System Diagnostic")
print("=" * 50)

# Environment info
print(f"🐍 Python: {sys.version}")
print(f"💻 Platform: {platform.platform()}")
print(f"📂 Working Dir: {os.getcwd()}")

# Check if in Colab
try:
    import google.colab
    print("☁️  Environment: Google Colab ✅")
    in_colab = True
except ImportError:
    print("💻 Environment: Local/Other")
    in_colab = False

# Memory info
try:
    import psutil
    memory = psutil.virtual_memory()
    print(f"💾 RAM: {memory.total / 1e9:.1f} GB total, {memory.available / 1e9:.1f} GB available")
except ImportError:
    print("💾 RAM: Memory info not available")

# GPU info with detailed status
try:
    import torch
    print(f"\n🎮 GPU Status:")
    print(f"   PyTorch: {torch.__version__}")
    print(f"   CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   GPU: {torch.cuda.get_device_name(0)}")
        print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        print(f"   CUDA Version: {torch.version.cuda}")
    else:
        print("   Mode: CPU only (fully functional)")
except ImportError:
    print("\n🎮 PyTorch: Not installed ❌")

# Package diagnostics
print(f"\n📦 Package Diagnostics:")
critical_packages = [
    'torch', 'torch_geometric', 'faiss', 'sentence_transformers', 
    'networkx', 'transformers', 'numpy', 'scikit_learn'
]

installed_packages = []
missing_packages = []

for package in critical_packages:
    try:
        if package == 'scikit_learn':
            import sklearn
            version = sklearn.__version__
            package_name = 'scikit-learn'
        elif package == 'faiss':
            import faiss
            version = getattr(faiss, '__version__', 'unknown')
            package_name = 'faiss'
        else:
            module = __import__(package)
            version = getattr(module, '__version__', 'unknown')
            package_name = package
            
        print(f"   ✅ {package_name}: {version}")
        installed_packages.append(package_name)
    except ImportError:
        print(f"   ❌ {package}: Not installed")
        missing_packages.append(package)

# Directory structure check
print(f"\n📁 Project Structure:")
if os.path.exists('src/insightspike'):
    print("   ✅ InsightSpike source code found")
    src_contents = os.listdir('src/insightspike')
    key_modules = ['core', 'utils', 'agents']
    for module in key_modules:
        if module in src_contents:
            print(f"   ✅ {module}/ module present")
        else:
            print(f"   ❌ {module}/ module missing")
else:
    print("   ❌ InsightSpike source code not found")
    print("       💡 Make sure you're in the correct directory")

# Configuration check
print(f"\n⚙️  Configuration Status:")
try:
    from src.insightspike.core.config import LLMConfig
    config = LLMConfig()
    print(f"   ✅ LLM Model: {config.model_name}")
    print(f"   ✅ Temperature: {config.temperature}")
except Exception as e:
    print(f"   ❌ Configuration error: {str(e)[:50]}...")

# Summary and recommendations
print(f"\n" + "=" * 50)
print(f"📊 SUMMARY:")
print(f"   Installed: {len(installed_packages)}/{len(critical_packages)} packages")
print(f"   Missing: {len(missing_packages)} packages")

if missing_packages:
    print(f"\n🔧 RECOMMENDED ACTIONS:")
    if 'torch_geometric' in missing_packages:
        print("   1. Install PyTorch Geometric: Re-run Step 2 dependency cell")
    if 'faiss' in missing_packages:
        print("   2. Install FAISS: !pip install faiss-gpu")
    if len(missing_packages) > 2:
        print("   3. Complete reinstall: Restart runtime, re-run all setup cells")
    print("   4. Check troubleshooting section above for specific solutions")
else:
    print(f"\n🎉 SYSTEM STATUS: Ready for research!")
    print(f"   All critical packages installed")
    print(f"   InsightSpike-AI is fully functional")

# Quick action buttons for Colab
if in_colab:
    print(f"\n🚀 QUICK ACTIONS:")
    print(f"   • Restart Runtime: Runtime > Restart Runtime")
    print(f"   • Re-run Setup: Execute cells 1-3 in order")
    print(f"   • Manual Fix: Use troubleshooting commands above")

print(f"\n" + "=" * 50)